**Table of contents**<a id='toc0_'></a>    
- [Good Seed Supermarket](#toc1_)    
  - [Initialization](#toc1_1_)    
  - [Load Data](#toc1_2_)    
  - [Conclusions](#toc1_3_)    
  - [EDA](#toc1_4_)    
    - [Findings](#toc1_4_1_)    
  - [Modelling](#toc1_5_)    
    - [Prepare the Script to Run on the GPU Platform](#toc1_5_1_)    
    - [Output](#toc1_5_2_)    
  - [Conclusions](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Good Seed Supermarket](#toc0_)

Good Seed Supermarket is looking for a data science solution to help them stay compliant with alcohol laws. They want to ensure that they are not selling alcohol to individuals under the legal limit. Good seed is equipped with cameras in the checkout area  which are triggered ehn a customer purchases alcohol. Given this information, our goal is to use computer vision to determine the age of a person from a picture. The target for the model is an MAE lower than 8.0. However, an MAE of 8 could mean that someone who is 13 years old could be incorrectly predicted to be 21. Therefore, our goal would be to minimize the MAE as much as possible to reduce the model's error. 

## <a id='toc1_1_'></a>[Initialization](#toc0_)

In [1]:
# !pip install plotly_express

## <a id='toc1_2_'></a>[Load Data](#toc0_)

In [2]:
# import libraries
import pandas as pd
import plotly_express as px
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
# read dataframe
#df = pd.read_csv('/datasets/faces/labels.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/faces/labels.csv'

In [ ]:
# look at dataframe
df.head()

In [ ]:
# summary on columns
df.info()

In [ ]:
# checking for missing values
df.isna().sum()

In [ ]:
# number of unique images
df.nunique()

## <a id='toc1_3_'></a>[Conclusions](#toc0_)

The dataframe contains the file name for the picture, and the age of the individual in the picture. The data is clean, with no missing values. Each image has a unique name, and there are 97 unique ages.  

## <a id='toc1_4_'></a>[EDA](#toc0_)

In [ ]:
# values for age 
age = df.real_age.value_counts()

In [ ]:
# distribution of age
age.describe()

In [ ]:
# distribution of age 
px.histogram(age, nbins=20)

In [ ]:
# boxplot of age
px.box(age)

In [ ]:
# look at a few images
# Get the first 15 rows of the DataFrame
df_head = df.sample(15)

# Iterate over each row in the DataFrame
for index, row in df_head.iterrows():
    # Load the image using matplotlib
    img = plt.imread(f"/datasets/faces/final_files/{row['file_name']}")

    # Display the image and the real age
    plt.imshow(img)
    plt.title(f"Age: {row['real_age']}")
    plt.show()


### <a id='toc1_4_1_'></a>[Findings](#toc0_)

The distribution of the data is distributed from 1 to 317, with  mean of 78.25 and a median of 54. The accuracy of the model would be improved if we reduced the amount of outliers in our data, and narrowed the age values from 5 to 99 years old. This represents a more reasonable distribution of age. 

The training data does not contain the best quality of images. The data contains images of cats, images that are cut off, and old film images that appear to have been scanned. the images are also in different scales, from various angles, and are taken with various light sources. Smaller pictures may not have enough details for our model to collect enough data points, thereby affecting the accuracy of our model. These poor quality images and irrelevant data may reduce the accuracy of the model in predicting human adults. 

## <a id='toc1_5_'></a>[Modelling](#toc0_)

In [ ]:
# function for loading training set
def load_train(path):
    labels = pd.read_csv('/datasets/faces/labels.csv')

    train_datagen = ImageDataGenerator(
        rescale=1./255, validation_split=0.25, vertical_flip=True, horizontal_flip=True)

    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        subset='training',
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=19)

    return train_gen_flow

In [ ]:
# function for loading test set
def load_test(path):
    labels = pd.read_csv('/datasets/faces/labels.csv')

    test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        subset='validation',
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=19)

    return test_gen_flow

In [ ]:
# function for creating model
def create_model(input_shape):

    backbone = ResNet50(
        input_shape=input_shape, weights='imagenet', include_top=False
    )

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(12, activation='relu'))

    model.compile(optimizer=Adam(learning_rate=0.00001),
                  loss='mse', metrics=['mae'])

    return model

In [ ]:
# function for training model
def train_model(
    model,
    train_data,
    test_data,
    batch_size=None,
    epochs=20,
    steps_per_epoch=None,
    validation_steps=None,
):
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(
        train_data,
        validation_data=test_data,
        batch_size=batch_size,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=2
    )

    return model

### <a id='toc1_5_1_'></a>[Prepare the Script to Run on the GPU Platform](#toc0_)

In [ ]:
# prepare a script to run on the GPU platform

init_str = """
import pandas as pd

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam



def load_train(path):
    labels = pd.read_csv('/datasets/faces/labels.csv')

    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25, vertical_flip=True, horizontal_flip=True)

    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        subset='training'
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=19)

    return train_gen_flow

def load_test(path):
    labels = pd.read_csv('/datasets/faces/labels.csv')
    
    test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        subset='testing'
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=19)
    
    return test_gen_flow


def create_model(input_shape):

    backbone = ResNet50(
    input_shape=input_shape, weights='imagenet', include_top=False
)

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(12, activation='relu'))

    model.compile(optimizer=Adam(learning_rate=0.00001), loss='mse', metrics=['mae'])

    return model  


def train_model(
    model,
    train_data,
    test_data,
    batch_size=None,
    epochs=20,
    steps_per_epoch=None,
    validation_steps=None,
):
    if steps_per_epoch is None:
            steps_per_epoch = len(train_data)
    if validation_steps is None:
            validation_steps = len(test_data)
            
    model.fit(
        train_data,
        validation_data=test_data,
        batch_size=batch_size,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=2
        )


    return model

"""

import inspect

with open('run_model_on_gpu.py', 'w') as f:
    
    f.write(init_str)
    f.write('\n\n')
        
    for fn_name in [load_train, load_test, create_model, train_model]:
        
        src = inspect.getsource(fn_name)
        f.write(src)
        f.write('\n\n')

### <a id='toc1_5_2_'></a>[Output](#toc0_)

2023-02-05 16:29:17.520285: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2023-02-05 16:29:17.521919: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
Using TensorFlow backend.
Found 5694 validated image filenames.
Found 1897 validated image filenames.
2023-02-05 16:29:18.538489: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-02-05 16:29:18.584901: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.585092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-02-05 16:29:18.585127: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-05 16:29:18.585160: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-05 16:29:18.587069: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-02-05 16:29:18.587392: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-02-05 16:29:18.589517: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-02-05 16:29:18.590737: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-02-05 16:29:18.590792: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-02-05 16:29:18.590875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.591090: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.591241: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2023-02-05 16:29:18.591566: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-05 16:29:18.598481: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300010000 Hz
2023-02-05 16:29:18.599112: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x40d2730 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-05 16:29:18.599136: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-02-05 16:29:18.685731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.686004: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x39e4fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-05 16:29:18.686027: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-02-05 16:29:18.686250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.686454: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-02-05 16:29:18.686496: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-05 16:29:18.686520: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-05 16:29:18.686556: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-02-05 16:29:18.686584: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-02-05 16:29:18.686601: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-02-05 16:29:18.686614: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-02-05 16:29:18.686625: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-02-05 16:29:18.686688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.686915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.687084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2023-02-05 16:29:18.687119: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-05 16:29:18.951960: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-02-05 16:29:18.952008: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 
2023-02-05 16:29:18.952018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N 
2023-02-05 16:29:18.952229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.952450: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.952647: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-02-05 16:29:18.952686: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14988 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:1e.0, compute capability: 7.0)
Downloading data from https://github.com/keras-team/keras-applications/releases/download/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5

    8192/94765736 [..............................] - ETA: 1s
12189696/94765736 [==>...........................] - ETA: 0s
24887296/94765736 [======>.......................] - ETA: 0s
37625856/94765736 [==========>...................] - ETA: 0s
50323456/94765736 [==============>...............] - ETA: 0s
62832640/94765736 [==================>...........] - ETA: 0s
75653120/94765736 [======================>.......] - ETA: 0s
88424448/94765736 [==========================>...] - ETA: 0s
94773248/94765736 [==============================] - 0s 0us/step
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 178 steps, validate for 60 steps
Epoch 1/20
2023-02-05 16:29:29.917139: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-05 16:29:30.151183: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
178/178 - 46s - loss: 1093.2465 - mae: 28.8412 - val_loss: 1127.3007 - val_mae: 29.0871
Epoch 2/20
178/178 - 38s - loss: 666.8450 - mae: 22.3902 - val_loss: 1131.4046 - val_mae: 29.1571
Epoch 3/20
178/178 - 38s - loss: 402.9212 - mae: 16.6575 - val_loss: 1110.2702 - val_mae: 28.9570
Epoch 4/20
178/178 - 38s - loss: 257.0143 - mae: 12.7257 - val_loss: 348.5501 - val_mae: 15.0817
Epoch 5/20
178/178 - 38s - loss: 175.7985 - mae: 10.1034 - val_loss: 152.9498 - val_mae: 9.4532
Epoch 6/20
178/178 - 38s - loss: 129.6226 - mae: 8.4551 - val_loss: 121.0458 - val_mae: 8.1466
Epoch 7/20
178/178 - 38s - loss: 103.0510 - mae: 7.3800 - val_loss: 122.3184 - val_mae: 8.2225
Epoch 8/20
178/178 - 38s - loss: 83.9488 - mae: 6.7479 - val_loss: 128.4400 - val_mae: 8.5485
Epoch 9/20
178/178 - 38s - loss: 67.9511 - mae: 6.1477 - val_loss: 87.1269 - val_mae: 6.9585
Epoch 10/20
178/178 - 38s - loss: 57.2124 - mae: 5.6546 - val_loss: 95.5864 - val_mae: 7.5535
Epoch 11/20
178/178 - 38s - loss: 47.7976 - mae: 5.2090 - val_loss: 89.6033 - val_mae: 7.0573
Epoch 12/20
178/178 - 38s - loss: 43.0968 - mae: 4.9940 - val_loss: 84.2371 - val_mae: 6.9016
Epoch 13/20
178/178 - 38s - loss: 39.1958 - mae: 4.7354 - val_loss: 84.4272 - val_mae: 6.8760
Epoch 14/20
178/178 - 38s - loss: 34.8503 - mae: 4.5421 - val_loss: 83.1303 - val_mae: 6.8225
Epoch 15/20
178/178 - 38s - loss: 31.8684 - mae: 4.3166 - val_loss: 91.4133 - val_mae: 7.3229
Epoch 16/20
178/178 - 38s - loss: 28.7383 - mae: 4.0954 - val_loss: 84.2958 - val_mae: 6.9114
Epoch 17/20
178/178 - 38s - loss: 25.7434 - mae: 3.8809 - val_loss: 85.1999 - val_mae: 6.8730
Epoch 18/20
178/178 - 38s - loss: 23.6390 - mae: 3.7272 - val_loss: 82.5083 - val_mae: 6.7850
Epoch 19/20
178/178 - 38s - loss: 22.8181 - mae: 3.6395 - val_loss: 85.5401 - val_mae: 6.8732
Epoch 20/20
178/178 - 38s - loss: 20.6016 - mae: 3.4828 - val_loss: 83.6391 - val_mae: 6.8220
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
60/60 - 9s - loss: 83.6391 - mae: 6.8220
Test MAE: 6.8220
2023-02-05 16:29:17.520285: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2023-02-05 16:29:17.521919: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
Using TensorFlow backend.
Found 5694 validated image filenames.
Found 1897 validated image filenames.
2023-02-05 16:29:18.538489: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-02-05 16:29:18.584901: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.585092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-02-05 16:29:18.585127: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-05 16:29:18.585160: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-05 16:29:18.587069: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-02-05 16:29:18.587392: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-02-05 16:29:18.589517: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-02-05 16:29:18.590737: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-02-05 16:29:18.590792: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-02-05 16:29:18.590875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.591090: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.591241: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2023-02-05 16:29:18.591566: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-05 16:29:18.598481: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300010000 Hz
2023-02-05 16:29:18.599112: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x40d2730 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-05 16:29:18.599136: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-02-05 16:29:18.685731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.686004: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x39e4fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-05 16:29:18.686027: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2023-02-05 16:29:18.686250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.686454: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-02-05 16:29:18.686496: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-05 16:29:18.686520: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-05 16:29:18.686556: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-02-05 16:29:18.686584: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-02-05 16:29:18.686601: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2023-02-05 16:29:18.686614: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2023-02-05 16:29:18.686625: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-02-05 16:29:18.686688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.686915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.687084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2023-02-05 16:29:18.687119: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-05 16:29:18.951960: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-02-05 16:29:18.952008: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 
2023-02-05 16:29:18.952018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N 
2023-02-05 16:29:18.952229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.952450: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 16:29:18.952647: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-02-05 16:29:18.952686: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14988 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:1e.0, compute capability: 7.0)
Downloading data from https://github.com/keras-team/keras-applications/releases/download/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5

    8192/94765736 [..............................] - ETA: 1s
12189696/94765736 [==>...........................] - ETA: 0s
24887296/94765736 [======>.......................] - ETA: 0s
37625856/94765736 [==========>...................] - ETA: 0s
50323456/94765736 [==============>...............] - ETA: 0s
62832640/94765736 [==================>...........] - ETA: 0s
75653120/94765736 [======================>.......] - ETA: 0s
88424448/94765736 [==========================>...] - ETA: 0s
94773248/94765736 [==============================] - 0s 0us/step
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 178 steps, validate for 60 steps
Epoch 1/20
2023-02-05 16:29:29.917139: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-05 16:29:30.151183: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
178/178 - 46s - loss: 1093.2465 - mae: 28.8412 - val_loss: 1127.3007 - val_mae: 29.0871
Epoch 2/20
178/178 - 38s - loss: 666.8450 - mae: 22.3902 - val_loss: 1131.4046 - val_mae: 29.1571
Epoch 3/20
178/178 - 38s - loss: 402.9212 - mae: 16.6575 - val_loss: 1110.2702 - val_mae: 28.9570
Epoch 4/20
178/178 - 38s - loss: 257.0143 - mae: 12.7257 - val_loss: 348.5501 - val_mae: 15.0817
Epoch 5/20
178/178 - 38s - loss: 175.7985 - mae: 10.1034 - val_loss: 152.9498 - val_mae: 9.4532
Epoch 6/20
178/178 - 38s - loss: 129.6226 - mae: 8.4551 - val_loss: 121.0458 - val_mae: 8.1466
Epoch 7/20
178/178 - 38s - loss: 103.0510 - mae: 7.3800 - val_loss: 122.3184 - val_mae: 8.2225
Epoch 8/20
178/178 - 38s - loss: 83.9488 - mae: 6.7479 - val_loss: 128.4400 - val_mae: 8.5485
Epoch 9/20
178/178 - 38s - loss: 67.9511 - mae: 6.1477 - val_loss: 87.1269 - val_mae: 6.9585
Epoch 10/20
178/178 - 38s - loss: 57.2124 - mae: 5.6546 - val_loss: 95.5864 - val_mae: 7.5535
Epoch 11/20
178/178 - 38s - loss: 47.7976 - mae: 5.2090 - val_loss: 89.6033 - val_mae: 7.0573
Epoch 12/20
178/178 - 38s - loss: 43.0968 - mae: 4.9940 - val_loss: 84.2371 - val_mae: 6.9016
Epoch 13/20
178/178 - 38s - loss: 39.1958 - mae: 4.7354 - val_loss: 84.4272 - val_mae: 6.8760
Epoch 14/20
178/178 - 38s - loss: 34.8503 - mae: 4.5421 - val_loss: 83.1303 - val_mae: 6.8225
Epoch 15/20
178/178 - 38s - loss: 31.8684 - mae: 4.3166 - val_loss: 91.4133 - val_mae: 7.3229
Epoch 16/20
178/178 - 38s - loss: 28.7383 - mae: 4.0954 - val_loss: 84.2958 - val_mae: 6.9114
Epoch 17/20
178/178 - 38s - loss: 25.7434 - mae: 3.8809 - val_loss: 85.1999 - val_mae: 6.8730
Epoch 18/20
178/178 - 38s - loss: 23.6390 - mae: 3.7272 - val_loss: 82.5083 - val_mae: 6.7850
Epoch 19/20
178/178 - 38s - loss: 22.8181 - mae: 3.6395 - val_loss: 85.5401 - val_mae: 6.8732
Epoch 20/20
178/178 - 38s - loss: 20.6016 - mae: 3.4828 - val_loss: 83.6391 - val_mae: 6.8220
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
60/60 - 9s - loss: 83.6391 - mae: 6.8220
Test MAE: 6.8220

## <a id='toc1_6_'></a>[Conclusions](#toc0_)

We effectively trained a model to verify the age of a customer from a photo. Our target metric was an MAE of 8, and were able to reduce it to 6.822 with our neural network. That represents nearly a 15% reduction in error from our target. This means our model is on average, off by 6.8 years when predicting a customer's age. Therefore, Good Seed can implement this model to enforce the alcohol purchasing laws and refrain from allowing minors to purchase alcohol.  